# Tarea: Recorrido por Colab

**JUAN SEBASTIAN GOMEZ DUQUE**

jgomez@unal.edu.co

1. Transcriba el siguiente código en Colab son sus propias manos.
2. Modifique el código para trabajar con GPU's
3. Modifique el código para trabajar con TPU's


## 1. Transcriba el siguiente código en Colab son sus propias manos.

### Lectura del set de datos

In [22]:

import numpy as np
import os
import gzip

def load_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
  with gzip.open(ruta_categorias, 'rb') as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
      imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas


In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Analisis de lenguaje natural/fashion-mnist-master/data/fashion'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Mounted at /content/gdrive


### Reajustar el tamaño de los datos

In [24]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

### Importar tensorflow 2 (ya incluye Keras)

In [25]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.8.0


In [26]:
tf.random.set_seed(200)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

### Entrenamiento con CPU

In [7]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None



In [8]:
with tf.device('/cpu:0'):
  model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)


Epoch 1/2
468/468 [==============================] - 804s 2s/step - loss: 0.6795 - accuracy: 0.7832 - val_loss: 0.3936 - val_accuracy: 0.8604
Epoch 2/2
468/468 [==============================] - 807s 2s/step - loss: 0.3922 - accuracy: 0.8611 - val_loss: 0.3333 - val_accuracy: 0.8853


In [14]:
checkpoint_path = "/content/gdrive/MyDrive/Analisis de lenguaje natural/final_model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [15]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


## 2. Modifique el código para trabajar con GPU's

In [16]:
def entrenamiento_gpu():
  with tf.device('/gpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

In [ ]:
entrenamiento_gpu()

Epoch 1/2
468/468 [==============================] - 816s 2s/step - loss: 0.3192 - accuracy: 0.8826 - val_loss: 0.2868 - val_accuracy: 0.9008
Epoch 2/2
436/468 [==========================>...] - ETA: 53s - loss: 0.2831 - accuracy: 0.8972

In [ ]:
save_path_2 = saver.save(model, "/content/gdrive/MyDrive/Analisis de lenguaje natural/final_model_2.ckpt")

In [18]:
checkpoint_path_2 = "/content/gdrive/MyDrive/Analisis de lenguaje natural/final_model_2.ckpt"
checkpoint_dir_2 = os.path.dirname(checkpoint_path_2)

In [19]:
cp_callback_2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_2,
                                                 save_weights_only=True,
                                                 verbose=1)

## 3. Modifique el código para trabajar con TPU's

In [30]:
def entrenamiento_tpu():
  with tf.device('/tpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

In [31]:
entrenamiento_tpu()

Epoch 1/2


InvalidArgumentError: ignored

In [ ]:
checkpoint_path_3 = "/content/gdrive/MyDrive/Analisis de lenguaje natural/final_model_3.ckpt"
checkpoint_dir_3 = os.path.dirname(checkpoint_path_3)